#   <center>Machine Learning Engineer Nanodegree</center>
##   <center>Project: Dogs vs. Cats Redux: Kernels Edition</center>
<center>
Author: Kyle Chen<br>
Date: 20180506<br>
Version: 20180506v1
</center>

---

###   写在前面
-   这次的毕业项目选做猫狗, 一方面是因为资料比较多, 资源比较丰富. 另外一方面, 图形识别也是当下的一大热点. 虽然毕业后可能会继续从事传统机器学习方面的研究, 但是能够有这段经历还是很不错的, 也能丰富自己的简历与深度学习的知识.
-   在这个项目中, 将通过评估几种模型与不同程度的调优, 来不断优化我们的模型框架, 以达到最终的top 10%标准.

###   准备数据
-   本文中的DataSet是已经从kaggle上拖取到本地, 并放入当前工作目录中, 但是由于Github的大小限制, 如果你想要执行并研究其中的一些代码, 请自行准备好数据集, 并存放到./DataSet目录下. kaggle传送门 <a href='https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data'>dogs-vs-cats-redux-kernels-edition/data</a>.
-   或者你可以直接运行下框的代码创建目录结构与拖取数据.

In [104]:
# 如果你已经拖取了数据, 请勿执行此代码框中的代码
!mkdir DataSet
!mkdir saved_models
!wget -c "https://storage.googleapis.com/kaggle-competitions-data/kaggle/5441/train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1526177720&Signature=CQ9z7GqLC5d%2BCuALIgPLIleOgpbls6dKQW6ykh8NBnaoiM%2F7MRC3dDSgiQWj5xxIr%2FHK4JXtCvBY75hLX4B2q9AW2c3l2r3z2IJpvb%2BeB0MXF7dO%2Fzbm0OmXNopMcI4wjdZYDLQuX5Sy69%2Fjc2W7rYohuo0QOT0vfofeHTEN3dmrwCkcPX7AZ3c2L1ByA%2FNnZtSGGGLGxLgNsYjQlZJNziM78RS0AQ5ztJUrQhEg8EevxPESN5lVZfvJOfNsP21%2FGOJOPWcllCyqqxPqv79l0WliatTG%2FMR1BfZ14bgX2f%2BEJ7b6W3omGpM7XRcBOm4vDvWclgivzwB2Dr7Xt7r7uw%3D%3D" -O DataSet/train.zip
!wget -c "https://storage.googleapis.com/kaggle-competitions-data/kaggle/5441/test.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1526177716&Signature=fHMWZ8zBkV0INrI2yVneHug3MdeZgjN2ViuucdAO7drp6iUQRZd09p3XzYL4tOy6sRL9syVk%2BSh1bf3V1qbB%2F1DAp%2BWK89GuiSHzeq0iUYi%2B8ylbUQtN2M7mQX2qsMMMr4b4bDIzOKqTkuYG8XzSQcEZWchY%2FoIHIDJjmm6nmJuu%2Bj%2F6K3jmaiL%2FnnJt1%2Firaa7l3Gys%2F4QxL9UaT6Lm%2BklbrnBqZR%2BX%2BdUMDPBXxhnBFbBQWB5ffECa50i8FKcJVtf1BO5u7Bolnx7EIwigCRsJO0gn%2BSvTLCKZYfSHuBdP2WLiSLJQXjPNjGZHx3tc8P9WEeHcQXvgdkXBRHqHVw%3D%3D" -O DataSet/test.zip
!wget -c "https://storage.googleapis.com/kaggle-competitions-data/kaggle/5441/sample_submission.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1526177711&Signature=P4cytvkXEgVM4tIUHsSr%2FhxsnUwC86EWXe0gYNuI%2Bu1Eyp7prfzCVZNPvnJ2VSidXKlxhQgtRrvyGjWcSXucimeWUzbErWhm0Wz%2BNVRDoPdxzhGhv1uT0eF1D81iemBurPEXD3b5q3glI1TJfRJEc3P3u2ObzPAsQrxuz76gm9ei2dnsUiweKNv2qPpr%2FogfaxI4EKtICdKkw33CMR%2FhBXZimr0L3Vs625AOj6HrfbGMTS2x8SiL48VAGLU%2BDhfOfoB0wJirHQogzHH7u0RKmcZJhqX9jP3UW3My1pevB3eIh8GND8xPQBlsJgJ3qJ7VMeMMTiyVctcb1B52EZ88Qw%3D%3D" -O DataSet/sample_submission.csv
!unzip DataSet/train.zip -d DataSet/
!unzip DataSet/test.zip -d DataSet/

--2018-05-10 03:12:08--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/5441/train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1526177720&Signature=CQ9z7GqLC5d%2BCuALIgPLIleOgpbls6dKQW6ykh8NBnaoiM%2F7MRC3dDSgiQWj5xxIr%2FHK4JXtCvBY75hLX4B2q9AW2c3l2r3z2IJpvb%2BeB0MXF7dO%2Fzbm0OmXNopMcI4wjdZYDLQuX5Sy69%2Fjc2W7rYohuo0QOT0vfofeHTEN3dmrwCkcPX7AZ3c2L1ByA%2FNnZtSGGGLGxLgNsYjQlZJNziM78RS0AQ5ztJUrQhEg8EevxPESN5lVZfvJOfNsP21%2FGOJOPWcllCyqqxPqv79l0WliatTG%2FMR1BfZ14bgX2f%2BEJ7b6W3omGpM7XRcBOm4vDvWclgivzwB2Dr7Xt7r7uw%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 569918665 (544M) [application/zip]
Saving to: ‘DataSet/train.zip’

DataSet/train.zip   100%[===================>] 543.52M   103MB/s    in 7.4s    

2018-05-10 03:12:16 (73.4 MB/s) - ‘Dat

  inflating: DataSet/train/cat.10189.jpg  
  inflating: DataSet/train/cat.1019.jpg  
  inflating: DataSet/train/cat.10190.jpg  
  inflating: DataSet/train/cat.10191.jpg  
  inflating: DataSet/train/cat.10192.jpg  
  inflating: DataSet/train/cat.10193.jpg  
  inflating: DataSet/train/cat.10194.jpg  
  inflating: DataSet/train/cat.10195.jpg  
  inflating: DataSet/train/cat.10196.jpg  
  inflating: DataSet/train/cat.10197.jpg  
  inflating: DataSet/train/cat.10198.jpg  
  inflating: DataSet/train/cat.10199.jpg  
  inflating: DataSet/train/cat.102.jpg  
  inflating: DataSet/train/cat.1020.jpg  
  inflating: DataSet/train/cat.10200.jpg  
  inflating: DataSet/train/cat.10201.jpg  
  inflating: DataSet/train/cat.10202.jpg  
  inflating: DataSet/train/cat.10203.jpg  
  inflating: DataSet/train/cat.10204.jpg  
  inflating: DataSet/train/cat.10205.jpg  
  inflating: DataSet/train/cat.10206.jpg  
  inflating: DataSet/train/cat.10207.jpg  
  inflating: DataSet/train/cat.10208.jpg  
  inflating: Da

  inflating: DataSet/train/cat.10404.jpg  
  inflating: DataSet/train/cat.10405.jpg  
  inflating: DataSet/train/cat.10406.jpg  
  inflating: DataSet/train/cat.10407.jpg  
  inflating: DataSet/train/cat.10408.jpg  
  inflating: DataSet/train/cat.10409.jpg  
  inflating: DataSet/train/cat.1041.jpg  
  inflating: DataSet/train/cat.10410.jpg  
  inflating: DataSet/train/cat.10411.jpg  
  inflating: DataSet/train/cat.10412.jpg  
  inflating: DataSet/train/cat.10413.jpg  
  inflating: DataSet/train/cat.10414.jpg  
  inflating: DataSet/train/cat.10415.jpg  
  inflating: DataSet/train/cat.10416.jpg  
  inflating: DataSet/train/cat.10417.jpg  
  inflating: DataSet/train/cat.10418.jpg  
  inflating: DataSet/train/cat.10419.jpg  
  inflating: DataSet/train/cat.1042.jpg  
  inflating: DataSet/train/cat.10420.jpg  
  inflating: DataSet/train/cat.10421.jpg  
  inflating: DataSet/train/cat.10422.jpg  
  inflating: DataSet/train/cat.10423.jpg  
  inflating: DataSet/train/cat.10424.jpg  
  inflating: 

  inflating: DataSet/train/cat.10611.jpg  
  inflating: DataSet/train/cat.10612.jpg  
  inflating: DataSet/train/cat.10613.jpg  
  inflating: DataSet/train/cat.10614.jpg  
  inflating: DataSet/train/cat.10615.jpg  
  inflating: DataSet/train/cat.10616.jpg  
  inflating: DataSet/train/cat.10617.jpg  
  inflating: DataSet/train/cat.10618.jpg  
  inflating: DataSet/train/cat.10619.jpg  
  inflating: DataSet/train/cat.1062.jpg  
  inflating: DataSet/train/cat.10620.jpg  
  inflating: DataSet/train/cat.10621.jpg  
  inflating: DataSet/train/cat.10622.jpg  
  inflating: DataSet/train/cat.10623.jpg  
  inflating: DataSet/train/cat.10624.jpg  
  inflating: DataSet/train/cat.10625.jpg  
  inflating: DataSet/train/cat.10626.jpg  
  inflating: DataSet/train/cat.10627.jpg  
  inflating: DataSet/train/cat.10628.jpg  
  inflating: DataSet/train/cat.10629.jpg  
  inflating: DataSet/train/cat.1063.jpg  
  inflating: DataSet/train/cat.10630.jpg  
  inflating: DataSet/train/cat.10631.jpg  
  inflating: 

  inflating: DataSet/train/cat.10823.jpg  
  inflating: DataSet/train/cat.10824.jpg  
  inflating: DataSet/train/cat.10825.jpg  
  inflating: DataSet/train/cat.10826.jpg  
  inflating: DataSet/train/cat.10827.jpg  
  inflating: DataSet/train/cat.10828.jpg  
  inflating: DataSet/train/cat.10829.jpg  
  inflating: DataSet/train/cat.1083.jpg  
  inflating: DataSet/train/cat.10830.jpg  
  inflating: DataSet/train/cat.10831.jpg  
  inflating: DataSet/train/cat.10832.jpg  
  inflating: DataSet/train/cat.10833.jpg  
  inflating: DataSet/train/cat.10834.jpg  
  inflating: DataSet/train/cat.10835.jpg  
  inflating: DataSet/train/cat.10836.jpg  
  inflating: DataSet/train/cat.10837.jpg  
  inflating: DataSet/train/cat.10838.jpg  
  inflating: DataSet/train/cat.10839.jpg  
  inflating: DataSet/train/cat.1084.jpg  
  inflating: DataSet/train/cat.10840.jpg  
  inflating: DataSet/train/cat.10841.jpg  
  inflating: DataSet/train/cat.10842.jpg  
  inflating: DataSet/train/cat.10843.jpg  
  inflating: 

...

  inflating: DataSet/test/9213.jpg   
  inflating: DataSet/test/9214.jpg   
  inflating: DataSet/test/9215.jpg   
  inflating: DataSet/test/9216.jpg   
  inflating: DataSet/test/9217.jpg   
  inflating: DataSet/test/9218.jpg   
  inflating: DataSet/test/9219.jpg   
  inflating: DataSet/test/922.jpg    
  inflating: DataSet/test/9220.jpg   
  inflating: DataSet/test/9221.jpg   
  inflating: DataSet/test/9222.jpg   
  inflating: DataSet/test/9223.jpg   
  inflating: DataSet/test/9224.jpg   
  inflating: DataSet/test/9225.jpg   
  inflating: DataSet/test/9226.jpg   
  inflating: DataSet/test/9227.jpg   
  inflating: DataSet/test/9228.jpg   
  inflating: DataSet/test/9229.jpg   
  inflating: DataSet/test/923.jpg    
  inflating: DataSet/test/9230.jpg   
  inflating: DataSet/test/9231.jpg   
  inflating: DataSet/test/9232.jpg   
  inflating: DataSet/test/9233.jpg   
  inflating: DataSet/test/9234.jpg   
  inflating: DataSet/test/9235.jpg   
  inflating: DataSet/test/9236.jpg   
  inflating:

  inflating: DataSet/test/9406.jpg   
  inflating: DataSet/test/9407.jpg   
  inflating: DataSet/test/9408.jpg   
  inflating: DataSet/test/9409.jpg   
  inflating: DataSet/test/941.jpg    
  inflating: DataSet/test/9410.jpg   
  inflating: DataSet/test/9411.jpg   
  inflating: DataSet/test/9412.jpg   
  inflating: DataSet/test/9413.jpg   
  inflating: DataSet/test/9414.jpg   
  inflating: DataSet/test/9415.jpg   
  inflating: DataSet/test/9416.jpg   
  inflating: DataSet/test/9417.jpg   
  inflating: DataSet/test/9418.jpg   
  inflating: DataSet/test/9419.jpg   
  inflating: DataSet/test/942.jpg    
  inflating: DataSet/test/9420.jpg   
  inflating: DataSet/test/9421.jpg   
  inflating: DataSet/test/9422.jpg   
  inflating: DataSet/test/9423.jpg   
  inflating: DataSet/test/9424.jpg   
  inflating: DataSet/test/9425.jpg   
  inflating: DataSet/test/9426.jpg   
  inflating: DataSet/test/9427.jpg   
  inflating: DataSet/test/9428.jpg   
  inflating: DataSet/test/9429.jpg   
  inflating:

  inflating: DataSet/test/9604.jpg   
  inflating: DataSet/test/9605.jpg   
  inflating: DataSet/test/9606.jpg   
  inflating: DataSet/test/9607.jpg   
  inflating: DataSet/test/9608.jpg   
  inflating: DataSet/test/9609.jpg   
  inflating: DataSet/test/961.jpg    
  inflating: DataSet/test/9610.jpg   
  inflating: DataSet/test/9611.jpg   
  inflating: DataSet/test/9612.jpg   
  inflating: DataSet/test/9613.jpg   
  inflating: DataSet/test/9614.jpg   
  inflating: DataSet/test/9615.jpg   
  inflating: DataSet/test/9616.jpg   
  inflating: DataSet/test/9617.jpg   
  inflating: DataSet/test/9618.jpg   
  inflating: DataSet/test/9619.jpg   
  inflating: DataSet/test/962.jpg    
  inflating: DataSet/test/9620.jpg   
  inflating: DataSet/test/9621.jpg   
  inflating: DataSet/test/9622.jpg   
  inflating: DataSet/test/9623.jpg   
  inflating: DataSet/test/9624.jpg   
  inflating: DataSet/test/9625.jpg   
  inflating: DataSet/test/9626.jpg   
  inflating: DataSet/test/9627.jpg   
  inflating:

  inflating: DataSet/test/980.jpg    
  inflating: DataSet/test/9800.jpg   
  inflating: DataSet/test/9801.jpg   
  inflating: DataSet/test/9802.jpg   
  inflating: DataSet/test/9803.jpg   
  inflating: DataSet/test/9804.jpg   
  inflating: DataSet/test/9805.jpg   
  inflating: DataSet/test/9806.jpg   
  inflating: DataSet/test/9807.jpg   
  inflating: DataSet/test/9808.jpg   
  inflating: DataSet/test/9809.jpg   
  inflating: DataSet/test/981.jpg    
  inflating: DataSet/test/9810.jpg   
  inflating: DataSet/test/9811.jpg   
  inflating: DataSet/test/9812.jpg   
  inflating: DataSet/test/9813.jpg   
  inflating: DataSet/test/9814.jpg   
  inflating: DataSet/test/9815.jpg   
  inflating: DataSet/test/9816.jpg   
  inflating: DataSet/test/9817.jpg   
  inflating: DataSet/test/9818.jpg   
  inflating: DataSet/test/9819.jpg   
  inflating: DataSet/test/982.jpg    
  inflating: DataSet/test/9820.jpg   
  inflating: DataSet/test/9821.jpg   
  inflating: DataSet/test/9822.jpg   
  inflating:

  inflating: DataSet/test/9993.jpg   
  inflating: DataSet/test/9994.jpg   
  inflating: DataSet/test/9995.jpg   
  inflating: DataSet/test/9996.jpg   
  inflating: DataSet/test/9997.jpg   
  inflating: DataSet/test/9998.jpg   
  inflating: DataSet/test/9999.jpg   


###   关于DataSet
-   先了解下数据集的组成.

In [105]:
!ls -ahl DataSet/

total 816M
drwxr-xr-x 4 root root 4.0K May 10 03:12 .
drwxr-xr-x 6 root root 4.0K May 10 03:12 ..
-rw-r--r-- 1 root root 112K Jun  8  2017 sample_submission.csv
drwxr-xr-x 2 root root 276K Sep 20  2013 test
-rw-r--r-- 1 root root 272M Jun  8  2017 test.zip
drwxr-xr-x 2 root root 740K Sep 20  2013 train
-rw-r--r-- 1 root root 544M Jun  8  2017 train.zip


-   test目录下存放的是kaggle准备好的测试集.
-   train目录下存放的是训练集, 当然, 还需要将其细分为训练集与验证集两个部分.

In [106]:
!ls -ahl DataSet/train/ | head -n 5

total 595M
drwxr-xr-x 2 root root 740K Sep 20  2013 .
drwxr-xr-x 4 root root 4.0K May 10 03:12 ..
-rw-r--r-- 1 root root  13K Sep 20  2013 cat.0.jpg
-rw-r--r-- 1 root root  22K Sep 20  2013 cat.10000.jpg
ls: write error: Broken pipe


-   发现数据集命名是有规则的, 遵循label.n.jpg的原则.

In [107]:
!echo "cats | $(find DataSet/train/ -name 'cat*' | wc -l)"
!echo "dogs | $(find DataSet/train/ -name 'dog*' | wc -l)"

cats | 12500
dogs | 12500


-   发现cats/dogs样本类型分布均匀.

###   Import Libs

In [108]:
# 导入我们后面需要用到的库
import os
import re
import h5py
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.utils import shuffle
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Lambda
from keras.layers import Input, Dropout, Flatten, Dense, BatchNormalization
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.applications.xception import Xception
from keras.preprocessing.image import ImageDataGenerator

###   Initial Global Variables

In [109]:
# initial global variables
TRAIN_DIR = 'DataSet/train'
TEST_DIR = 'DataSet/test'
TRANSFER_DIR = 'DataSet/transfer'
TRANSFER_TRAIN_DIR = 'DataSet/transfer/train'
TRANSFER_TEST_DIR = 'DataSet/transfer/test'
RESULT_SAMP = 'DataSet/sample_submission.csv'
RESULT_FILE = 'DataSet/result.csv'
np.random.seed(2018)

###   导入数据集

In [110]:
# load data from TRAIN_DIR
def load_data(width, height, channels):
    img_list = os.listdir(TRAIN_DIR)
    nums = len(img_list)
    data = np.empty((nums, width, height, channels), dtype="float32")
    label = np.empty((nums, ))
    
    i = 0
    while i < nums:
        img = img_list[i]
        imgObj = Image.open("{}/{}".format(TRAIN_DIR, img))
        arr = np.asarray(imgObj, dtype="float32")
        arr.resize((width, height, channels))
        data[i, :, :, :] = arr
        
        if re.match(r'^cat\.', img) != None:
            label[i] = 0
            
        elif re.match(r'^dog\.', img) != None:
            label[i] = 1            
        i += 1
    return(data, label)

###   CNN模型框架
-   Load数据

In [111]:
# run load data
data, label = load_data(224, 224, 3)
data, label = shuffle(data, label)

-   CNN模型设计

In [112]:
cnn_model = Sequential()
shape_input = (len(data[0]), len(data[0][0]), len(data[0][0][0]))
cnn_model.add(Conv2D(filters=16, kernel_size=2, input_shape=shape_input))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=2, padding='valid'))
cnn_model.add(Dense(133, activation='relu'))
cnn_model.add(Conv2D(filters=32, kernel_size=2))
cnn_model.add(MaxPooling2D(pool_size=2, padding='valid'))
cnn_model.add(Dense(133, activation='relu'))
cnn_model.add(Conv2D(filters=64, kernel_size=2))
cnn_model.add(MaxPooling2D(pool_size=2, padding='valid'))
cnn_model.add(GlobalAveragePooling2D(dim_ordering='default'))
cnn_model.add(Dense(1, activation='sigmoid'))
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 223, 223, 16)      208       
_________________________________________________________________
batch_normalization_22 (Batc (None, 223, 223, 16)      64        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
dense_6 (Dense)              (None, 111, 111, 133)     2261      
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 110, 110, 32)      17056     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 55, 55, 32)        0         
_________________________________________________________________
dense_7 (Dense)              (None, 55, 55, 133)       4389      
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `GlobalAveragePooling2D` call to the Keras 2 API: `GlobalAveragePooling2D(data_format=None)`
  if sys.path[0] == '':


-   编译模型

In [113]:
# 编译模型
# 在keras中, 提供binary_crossentropy函数, 就是我们需要的LogLoss
cnn_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

-   训练CNN模型

In [114]:
# 这里在8c 16g的mbpr上跑了差不多两个半小时
epochs = 10
batch_size = 128
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.cnn.hdf5', 
                                   verbose=1, save_best_only=True)
cnn_model.fit(data, label, validation_split = 0.3,
                epochs = epochs, batch_size = batch_size,
                callbacks=[checkpointer], verbose=1)

Train on 17500 samples, validate on 7500 samples
Epoch 1/10
17500/17500 [==============================] - 49s 3ms/step - loss: 0.6792 - acc: 0.5739 - val_loss: 0.6837 - val_acc: 0.5567

Epoch 00001: val_loss improved from inf to 0.68374, saving model to saved_models/weights.best.cnn.hdf5
Epoch 2/10
17500/17500 [==============================] - 46s 3ms/step - loss: 0.6596 - acc: 0.6019 - val_loss: 0.6602 - val_acc: 0.6004

Epoch 00002: val_loss improved from 0.68374 to 0.66016, saving model to saved_models/weights.best.cnn.hdf5
Epoch 3/10
17500/17500 [==============================] - 46s 3ms/step - loss: 0.6509 - acc: 0.6184 - val_loss: 0.6549 - val_acc: 0.6011

Epoch 00003: val_loss improved from 0.66016 to 0.65492, saving model to saved_models/weights.best.cnn.hdf5
Epoch 4/10
17500/17500 [==============================] - 46s 3ms/step - loss: 0.6437 - acc: 0.6260 - val_loss: 0.6548 - val_acc: 0.5988

Epoch 00004: val_loss improved from 0.65492 to 0.65480, saving model to saved_mode

-   这里可以发现, 用CNN优化之后, 还是在0.63894. LogLoss没有太大的提升.

###   使用Xception
-   将train下的数据分类至transfer目录下

In [115]:
# 将train下的数据分类并拷贝至transfer/{cat,dog}目录下
!mkdir DataSet/transfer/train/cat -p
!mkdir DataSet/transfer/train/dog
!mkdir DataSet/transfer/test/pic -p
!find DataSet/train -name 'cat.*' -exec cp -rf {} DataSet/transfer/train/cat/ \;
!find DataSet/train -name 'dog.*' -exec cp -rf {} DataSet/transfer/train/dog/ \;
!find DataSet/test -name '*.jpg' -exec cp -rf {} DataSet/transfer/test/pic/ \;

-   加载数据

In [116]:
# run load data, change size to Xception default (299,299)
img_size = (299, 299)
gen = ImageDataGenerator()
X_train_gen = gen.flow_from_directory(TRANSFER_TRAIN_DIR, img_size, shuffle = False, 
                                        batch_size = 16)
X_test_gen = gen.flow_from_directory(TRANSFER_TEST_DIR, img_size, shuffle = False, 
                                        batch_size = 16, classes = None)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


-   训练模型设计

In [117]:
input_tensor = Input((img_size[0], img_size[1], 3))
input_tensor = Lambda(xception.preprocess_input)(input_tensor)
Xception_base = Xception(input_tensor = input_tensor, 
                             weights = 'imagenet', include_top = False)
Xception_model = Model(Xception_base.input, GlobalAveragePooling2D()(Xception_base.output))
Xception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 299, 299, 3)  0           input_13[0][0]                   
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_3[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

-   利用Xception训练特征向量

In [118]:
X_train = Xception_model.predict_generator(X_train_gen, verbose=1)
X_test = Xception_model.predict_generator(X_test_gen, verbose=1)

782/782 [==============================] - 91s 116ms/step


-   保存特征权重

In [119]:
with h5py.File('saved_models/weights.Xception.hdf5') as fp:
        fp.create_dataset('train', data = X_train)
        fp.create_dataset('test', data = X_test)
        fp.create_dataset('label', data = X_train_gen.classes)

-   导入我们刚刚训练好的特征向量

In [120]:
X_train = []
X_test = []
with h5py.File('saved_models/weights.Xception.hdf5', 'r') as fp:
    X_train.append(np.array(fp['train']))
    X_test.append(np.array(fp['test']))
    y_train = np.array(fp['label'])
    
X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)
X_train, y_train = shuffle(X_train, y_train)

-   使用Xception训练好的特征向量构建模型

In [121]:
input_tensor = Input(X_train.shape[1:])
Xception_model = Model(input_tensor, Dropout(0.5)(input_tensor))
Xception_model = Model(Xception_model.input, Dense(1, activation = 'sigmoid')(Xception_model.output))
Xception_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 2048)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 2049      
Total params: 2,049
Trainable params: 2,049
Non-trainable params: 0
_________________________________________________________________


-   编译模型

In [122]:
Xception_model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

-   使用Xception训练模型

In [123]:
epochs = 10
batch_size = 128
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.Xception.hdf5', 
                                   verbose=1, save_best_only=True)
Xception_model.fit(X_train, y_train, validation_split = 0.3,
                    epochs = epochs, batch_size = batch_size, verbose=1,
                    callbacks=[checkpointer])

Train on 17500 samples, validate on 7500 samples
Epoch 1/10
17500/17500 [==============================] - 3s 197us/step - loss: 0.1558 - acc: 0.9758 - val_loss: 0.0518 - val_acc: 0.9907

Epoch 00001: val_loss improved from inf to 0.05176, saving model to saved_models/weights.best.Xception.hdf5
Epoch 2/10
17500/17500 [==============================] - 1s 36us/step - loss: 0.0436 - acc: 0.9919 - val_loss: 0.0291 - val_acc: 0.9928

Epoch 00002: val_loss improved from 0.05176 to 0.02914, saving model to saved_models/weights.best.Xception.hdf5
Epoch 3/10
17500/17500 [==============================] - 1s 36us/step - loss: 0.0299 - acc: 0.9927 - val_loss: 0.0229 - val_acc: 0.9936

Epoch 00003: val_loss improved from 0.02914 to 0.02293, saving model to saved_models/weights.best.Xception.hdf5
Epoch 4/10
17500/17500 [==============================] - 1s 36us/step - loss: 0.0264 - acc: 0.9928 - val_loss: 0.0209 - val_acc: 0.9936

Epoch 00004: val_loss improved from 0.02293 to 0.02090, saving mod

-   可以发现, 使用Xception比直接使用CNN的表现要好很多.
###   尝试往kaggle上提交


-   对测试集进行预测

In [124]:
y_test = Xception_model.predict(X_test)
y_test = y_test.clip(min = 0.005, max = 0.995)

-   保存数据

In [125]:
num = [ int(re.sub('.jpg', '', re.sub('.*\/', '', f))) for f in X_test_gen.filenames ]
df = pd.read_csv(RESULT_SAMP)
for i in range(len(num)):
    df.set_value(num[i] - 1, 'label', y_test[i])
df.to_csv(RESULT_FILE, index=None)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
